In [10]:
import numpy as np

In [16]:
x = [[1,2],[3,4]]
z = x
y = np.square(x)
z = y
y = 2*y + z
y


array([[ 3, 12],
       [27, 48]])

In [27]:
import numpy as np
import cv2  # Still used for image I/O and adding noise
import matplotlib.pyplot as plt
import time

u = [[1,2],[3,4]]
# Fourier Transformation Utility Functions using NumPy
def fft2(u):
##    return np.fft.fftshift(np.fft.fft2(u))
    M,N = u.shape
    return np.fft.fft2(u)

def ifft2(u_hat):
##    return np.fft.ifft2(np.fft.ifftshift(u_hat)).real
    #M,N = u_hat.shape
    return  np.fft.ifft2(u_hat).real
    #return np.ifft2(u_hat).real

# Backward and Central Fractional Derivatives
# def back_Dx(u, alpha):
#     u_hat = fft2(u)
#     M, N = u.shape
#     w1_grid = np.fft.fftshift(np.fft.fftfreq(M) * M)
#     complex_term = 1 - np.exp((-1j * 2 * np.pi * w1_grid[:, None]) / M)
#     K = np.power(complex_term, alpha)
#     dx_hat = u_hat * K
#     dx = ifft2(dx_hat)
#     return dx

# def back_Dy(u, alpha):
#     u_hat = fft2(u)
#     M, N = u.shape
#     w2_grid = np.fft.fftshift(np.fft.fftfreq(N) * N)
#     complex_term = 1 - np.exp((-1j * 2 * np.pi * w2_grid) / N)
#     K = np.power(complex_term, alpha)
#     dy_hat = u_hat * K
#     dy = ifft2(dy_hat)
#     return dy

def _Dx(u, alpha):
    u_hat = fft2(u)
    #M, N = u.shape
    M,N = 2
    w1_grid = np.fft.fftshift(np.fft.fftfreq(M) * M)
    complex_term = np.exp((1j * np.pi * w1_grid[:, None]) / M) - np.exp((-1j * np.pi * w1_grid[:, None]) / M)
    K = np.power(complex_term, alpha)
    dx_hat = u_hat * K
    dx = ifft2(dx_hat)
    return dx

def Dx(u, alpha):
    u_hat = fft2(u)
    # M, N = u.shape
    M,N = 2
    array_x, array_y = np.mgrid[0:M,0:N]
    x = np.power(1 - np.exp(-1j*2*np.pi * array_x/M), alpha)
    ##K = np.power(complex_term, alpha)
    dx_hat = u_hat * x
    dx = ifft2(dx_hat)
    return dx

def _Dy(u, alpha):
    u_hat = fft2(u)
    M, N = u.shape
    w2_grid = np.fft.fftshift(np.fft.fftfreq(N) * N)
    complex_term = np.exp((1j * np.pi * w2_grid) / N) - np.exp((-1j * np.pi * w2_grid) / N)
    K = np.power(complex_term, alpha)
    dy_hat = u_hat * K
    dy = ifft2(dy_hat)
    return dy

def Dy(u, alpha):
    u_hat = fft2(u)
    M, N = u.shape
    array_x, array_y = np.mgrid[0:M,0:N]

    y = np.power(1 - np.exp(-1j*2*np.pi * array_y/N), alpha)
    ##K = np.power(complex_term, alpha)
    dy_hat = u_hat * y
    dy = ifft2(dy_hat)
    return dy


# Diffusion Coefficient Computation
def C(u, alpha, beta):
    D = np.sqrt(np.square(Dx(u, alpha)) + np.square(Dy(u, alpha)))
    mu_nn_numerator = np.square(Dx(u, beta)) * Dx(Dx(u, beta), beta) + 2 * Dx(u, beta) * Dy(u, beta) * Dx(Dy(u, beta), beta) + np.square(Dy(u, beta)) * Dy(Dy(u, beta), beta)
    mu_nn_denominator = np.square(Dx(u, beta)) + np.square(Dy(u, beta)) + 1000000000
    mu_nn = mu_nn_numerator / mu_nn_denominator
    mu_ee_numerator = np.square(Dx(u, beta)) * Dy(Dy(u, beta), beta) - 2 * Dx(u, beta) * Dy(u, beta) * Dx(Dy(u, beta), beta) + np.square(Dy(u, beta)) * Dx(Dx(u, beta), beta)
    mu_ee_denominator = np.square(Dx(u, beta)) + np.square(Dy(u, beta)) + 1000000000
    mu_ee = mu_ee_numerator / mu_ee_denominator
    fDC = np.abs(np.abs(mu_nn) - np.abs(mu_ee))
    c = 1 / (1 + D +fDC)
    return c

# Compute Kx and Ky using NumPy
# def make_K(M, N, alpha):
#     w1_grid = np.fft.fftshift(np.fft.fftfreq(M) * M)
#     w2_grid = np.fft.fftshift(np.fft.fftfreq(N) * N)
#     complex_term_x = np.exp((1j * np.pi * w1_grid[:, None]) / M) - np.exp((-1j * np.pi * w1_grid[:, None]) / M)
#     complex_term_y = np.exp((1j * np.pi * w2_grid) / N) - np.exp((-1j * np.pi * w2_grid) / N)
#     Kx = np.power(complex_term_x, alpha)
#     Ky = np.power(complex_term_y, alpha)
#     return Kx, Ky

def make_K(M, N, alpha):

    diagonal_matrix = np.diag(np.arange(M))
    #w1_grid = np.fft.fftshift(np.fft.fftfreq(M) * M)
    #w2_grid = np.fft.fftshift(np.fft.fftfreq(N) * N)
    #complex_term_x = np.exp((1j * np.pi * w1_grid[:, None]) / M) - np.exp((-1j * np.pi * w1_grid[:, None]) / M)
    #complex_term_y = np.exp((1j * np.pi * w2_grid) / N) - np.exp((-1j * np.pi * w2_grid) / N)
    Kx = np.power(1 - np.exp(-1j*2*np.pi * diagonal_matrix/M), alpha)*np.power(np.exp(1j*np.pi * diagonal_matrix/M), alpha)
    Ky = np.power(1 - np.exp(-1j*2*np.pi * diagonal_matrix/M), alpha)*np.power(np.exp(1j*np.pi * diagonal_matrix/M), alpha)
    return Kx, Ky

In [29]:
array_x, array_y = np.mgrid[0:10,0:10]
array_x
1/(1+array_x)

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ],
       [0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333],
       [0.25      , 0.25      , 0.25      , 0.25      , 0.25      ,
        0.25      , 0.25      , 0.25      , 0.25      , 0.25      ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
        0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667],
       [0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
        0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714],
       [0.125     , 0.125     , 0.125    

In [31]:
x = [[1,1],[1,1]]
np.fft.fft2(x)
np.fft.ifft2(np.fft.fft2(x))

array([[1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j]])

In [26]:
fft2(u)
ifft2(u)
Dx(u,1)

TypeError: cannot unpack non-iterable int object

In [42]:
import numpy as np
import cv2  # Still used for image I/O and adding noise
import matplotlib.pyplot as plt
import time

# Fourier Transformation Utility Functions using NumPy
def fft2(u):
##    return np.fft.fftshift(np.fft.fft2(u))
   # M,N = u.shape
    M = 2
    N = 2
    return np.fft.fft2(u)/(M)

def ifft2(u_hat):
##    return np.fft.ifft2(np.fft.ifftshift(u_hat)).real
    #M,N = u_hat.shape
    M = 2
    N = 2
    return  M*np.fft.ifft2(u_hat)
    #return np.ifft2(u_hat).real

def Dx_central(u, alpha):
    u_hat = fft2(u)
    #M, N = u.shape
    M = 2
    N = 2
    array_x, array_y = np.mgrid[0:M,0:N]

    x = np.power(1 - np.exp(-1j*2*np.pi * array_x/M), alpha)
    z = np.power(np.exp(1j*np.pi * array_x/M), alpha)
    ##K = np.power(complex_term, alpha)
    dx_hat = u_hat * x * z
    dx = ifft2(dx_hat)
    return dx

def Dy_central(u, alpha):
    u_hat = fft2(u)
    #M, N = u.shape
    M = 2
    N = 2
    array_x, array_y = np.mgrid[0:M,0:N]

    y = np.power(1 - np.exp(-1j*2*np.pi * array_y/N), alpha)
    z = np.power(np.exp(1j*np.pi * array_y/M), alpha)
    ##K = np.power(complex_term, alpha)
    dy_hat = u_hat * y * z
    dy = ifft2(dy_hat)
    return dy

def C(u, alpha, beta):
    D = (np.square(Dx_central(u, alpha)) + np.square(Dy_central(u, alpha)))
    c = 1 / (1 + D)
    return c

def make_K(M, N, alpha):

    diagonal_matrix = np.diag(np.arange(M))
    Kx = np.power(1 - np.exp(-1j*2*np.pi * diagonal_matrix/M), alpha)*np.power(np.exp(1j*np.pi * diagonal_matrix/M), alpha)
    Ky = np.power(1 - np.exp(-1j*2*np.pi * diagonal_matrix/M), alpha)*np.power(np.exp(1j*np.pi * diagonal_matrix/M), alpha)
    return Kx, Ky

In [43]:
x = [[1,1],[1,1]]
print(fft2(x))
print(fft2(fft2(x)))
print(Dx_central(x,2))
print(Dy_central(x,2))
print(C(x,2,1))
print(make_K(2,2,2))

[[2.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 1.+0.j]
 [1.+0.j 1.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 1.+0.j]
 [1.+0.j 1.+0.j]]
(array([[ 0.+0.j,  0.+0.j],
       [ 0.+0.j, -4.+0.j]]), array([[ 0.+0.j,  0.+0.j],
       [ 0.+0.j, -4.+0.j]]))


In [48]:
array_x, array_y = np.mgrid[0:10,0:10]
z = np.power(np.exp(1j*np.pi * array_x/10), 1)
print(z)
ifft2(z)

[[ 1.00000000e+00+0.j          1.00000000e+00+0.j
   1.00000000e+00+0.j          1.00000000e+00+0.j
   1.00000000e+00+0.j          1.00000000e+00+0.j
   1.00000000e+00+0.j          1.00000000e+00+0.j
   1.00000000e+00+0.j          1.00000000e+00+0.j        ]
 [ 9.51056516e-01+0.30901699j  9.51056516e-01+0.30901699j
   9.51056516e-01+0.30901699j  9.51056516e-01+0.30901699j
   9.51056516e-01+0.30901699j  9.51056516e-01+0.30901699j
   9.51056516e-01+0.30901699j  9.51056516e-01+0.30901699j
   9.51056516e-01+0.30901699j  9.51056516e-01+0.30901699j]
 [ 8.09016994e-01+0.58778525j  8.09016994e-01+0.58778525j
   8.09016994e-01+0.58778525j  8.09016994e-01+0.58778525j
   8.09016994e-01+0.58778525j  8.09016994e-01+0.58778525j
   8.09016994e-01+0.58778525j  8.09016994e-01+0.58778525j
   8.09016994e-01+0.58778525j  8.09016994e-01+0.58778525j]
 [ 5.87785252e-01+0.80901699j  5.87785252e-01+0.80901699j
   5.87785252e-01+0.80901699j  5.87785252e-01+0.80901699j
   5.87785252e-01+0.80901699j  5.87785252e-

array([[ 2.00000000e-01+1.26275030e+00j,  0.00000000e+00+0.00000000e+00j,
        -8.88178420e-18+1.55431223e-17j,  0.00000000e+00+0.00000000e+00j,
         3.08148791e-34+1.33226763e-17j,  0.00000000e+00+0.00000000e+00j,
         3.08148791e-34+1.33226763e-17j,  0.00000000e+00+0.00000000e+00j,
        -8.88178420e-18+1.55431223e-17j,  0.00000000e+00+0.00000000e+00j],
       [ 2.00000000e-01+3.92522101e-01j,  0.00000000e+00+0.00000000e+00j,
         4.38830251e-18+4.32311530e-20j,  0.00000000e+00+0.00000000e+00j,
         8.44257447e-18+3.77103311e-18j,  0.00000000e+00+0.00000000e+00j,
         8.44257447e-18+3.77103311e-18j,  0.00000000e+00+0.00000000e+00j,
         4.38830251e-18+4.32311530e-20j,  0.00000000e+00+0.00000000e+00j],
       [ 2.00000000e-01+2.00000000e-01j,  0.00000000e+00+0.00000000e+00j,
        -8.95460846e-18-2.01524432e-17j,  0.00000000e+00+0.00000000e+00j,
        -9.62446745e-18-1.07156101e-17j,  0.00000000e+00+0.00000000e+00j,
        -9.62446745e-18-1.07156101e-

In [55]:
w1_grid = np.fft.fftshift(np.fft.fftfreq(220)*220)
array_x, array_y = np.mgrid[0:220,0:220]
print(w1_grid)
z = np.ones(220*200)
w1_grid[219]

[-110. -109. -108. -107. -106. -105. -104. -103. -102. -101. -100.  -99.
  -98.  -97.  -96.  -95.  -94.  -93.  -92.  -91.  -90.  -89.  -88.  -87.
  -86.  -85.  -84.  -83.  -82.  -81.  -80.  -79.  -78.  -77.  -76.  -75.
  -74.  -73.  -72.  -71.  -70.  -69.  -68.  -67.  -66.  -65.  -64.  -63.
  -62.  -61.  -60.  -59.  -58.  -57.  -56.  -55.  -54.  -53.  -52.  -51.
  -50.  -49.  -48.  -47.  -46.  -45.  -44.  -43.  -42.  -41.  -40.  -39.
  -38.  -37.  -36.  -35.  -34.  -33.  -32.  -31.  -30.  -29.  -28.  -27.
  -26.  -25.  -24.  -23.  -22.  -21.  -20.  -19.  -18.  -17.  -16.  -15.
  -14.  -13.  -12.  -11.  -10.   -9.   -8.   -7.   -6.   -5.   -4.   -3.
   -2.   -1.    0.    1.    2.    3.    4.    5.    6.    7.    8.    9.
   10.   11.   12.   13.   14.   15.   16.   17.   18.   19.   20.   21.
   22.   23.   24.   25.   26.   27.   28.   29.   30.   31.   32.   33.
   34.   35.   36.   37.   38.   39.   40.   41.   42.   43.   44.   45.
   46.   47.   48.   49.   50.   51.   52.   53.   

109.0

In [78]:

    
size = 10
middle_row_index = size//2
matrix = np.zeros((size, size), dtype=int)
# if n is odd
for i in range(middle_row_index):
    if (size%2) :
        matrix[:,i] = - size + i +1 + middle_row_index
    else :
        matrix[:,i] = - size + i + middle_row_index
for i in range(middle_row_index + 1, size):
    matrix[:,i] = + i - middle_row_index   

print(matrix)


[[-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]
 [-5 -4 -3 -2 -1  0  1  2  3  4]]


In [88]:
x = np.array([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]])
y = np.fft.fftshift(np.fft.fft2(x))
z = np.fft.fft2(x)
# print(y)
# print(z)
a = np.power(1 - np.exp(-1j*2*np.pi * x/2), 2)
b = np.power(np.exp(1j*np.pi * x/2), 2)
print(a)
print(b)


[[4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j]
 [4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j]
 [4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j]
 [4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j 4.+4.8985872e-16j]]
[[-1.+1.2246468e-16j -1.+1.2246468e-16j -1.+1.2246468e-16j
  -1.+1.2246468e-16j]
 [-1.+1.2246468e-16j -1.+1.2246468e-16j -1.+1.2246468e-16j
  -1.+1.2246468e-16j]
 [-1.+1.2246468e-16j -1.+1.2246468e-16j -1.+1.2246468e-16j
  -1.+1.2246468e-16j]
 [-1.+1.2246468e-16j -1.+1.2246468e-16j -1.+1.2246468e-16j
  -1.+1.2246468e-16j]]


In [91]:
diagonal_matrix = np.diag(np.arange(220))
diagonal_matrix = diagonal_matrix - 110*np.eye(220)
print(diagonal_matrix)

[[-110.    0.    0. ...    0.    0.    0.]
 [   0. -109.    0. ...    0.    0.    0.]
 [   0.    0. -108. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ...  107.    0.    0.]
 [   0.    0.    0. ...    0.  108.    0.]
 [   0.    0.    0. ...    0.    0.  109.]]
